## Graph programs

The Graph Programs are a special data type representing a workflow of actions and decisions with calls to other programs. They are used by our own custom Agent, the `GraphProgramInterpreter`. In order help you to build them, we provide two ways of doing it: Using Python or Cypher.

In [1]:
import hybridagi.core.graph_program as gp

main = gp.GraphProgram(
	name = "main",
	description = "The main program",
)

main.add(gp.Action(
    id = "answer",
	tool = "Speak",
	purpose = "Answer the user question",
	prompt = "Please answer to the Objective's question",
))

main.connect("start", "answer")
main.connect("answer", "end")

# We build the program to verify its structure
main.build()

# Let's see what this program look like
print(main)

/home/yoan/.cache/pypoetry/virtualenvs/hybridagi-B1GoJrSC-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


// @desc: The main program
CREATE
// Nodes declaration
(start:Control {id: "start"}),
(end:Control {id: "end"}),
(answer:Action {
  id: "answer",
  purpose: "Answer the user question",
  tool: "Speak",
  prompt: "Please answer to the Objective's question"
}),
// Structure declaration
(start)-[:NEXT]->(answer),
(answer)-[:NEXT]->(end)


In [2]:
# Or we can visualize it using pyvis

# Use notebook=False if you are running in a local settings and want to open a browser page
main.show()

# You can even save it as a cypher file

main.save()

main.html


### Using decision-making steps

Decision making steps allow the model to branch over different paths in a program, like conditions in traditional programming, they allow conditional loops and multi-output decisions.

In [3]:
main = gp.GraphProgram(
    name="main",
    description="The main program",
)
    
main.add(gp.Decision(
    id="is_objective_unclear",
    purpose="Check if the Objective's is unclear",
    question="Is the Objective's question unclear?",
))

main.add(gp.Action(
    id="clarify",
    purpose="Ask one question to clarify the user's Objective",
    tool="AskUser",
    prompt="Please pick one question to clarify the Objective's question",
))

main.add(gp.Action(
    id="answer",
    purpose="Answer the question",
    tool="Speak",
    prompt="Please answer to the Objective's question",
))
    
main.add(gp.Action(
    id="refine_objective",
    purpose="Refine the objective",
    tool="UpdateObjective",
    prompt="Please refine the user Objective",
))
    
main.connect("start", "is_objective_unclear")
main.connect("is_objective_unclear", "clarify", label="Clarify")
main.connect("is_objective_unclear", "answer", label="Answer")
main.connect("clarify", "refine_objective")
main.connect("refine_objective", "answer")
main.connect("answer", "end")

main.build()

print(main)

// @desc: The main program
CREATE
// Nodes declaration
(start:Control {id: "start"}),
(end:Control {id: "end"}),
(is_objective_unclear:Decision {
  id: "is_objective_unclear",
  purpose: "Check if the Objective's is unclear",
  question: "Is the Objective's question unclear?"
}),
(clarify:Action {
  id: "clarify",
  purpose: "Ask one question to clarify the user's Objective",
  tool: "AskUser",
  prompt: "Please pick one question to clarify the Objective's question"
}),
(answer:Action {
  id: "answer",
  purpose: "Answer the question",
  tool: "Speak",
  prompt: "Please answer to the Objective's question"
}),
(refine_objective:Action {
  id: "refine_objective",
  purpose: "Refine the objective",
  tool: "UpdateObjective",
  prompt: "Please refine the user Objective"
}),
// Structure declaration
(start)-[:NEXT]->(is_objective_unclear),
(is_objective_unclear)-[:CLARIFY]->(clarify),
(is_objective_unclear)-[:ANSWER]->(answer),
(clarify)-[:NEXT]->(refine_objective),
(answer)-[:NEXT]->(end),

In [4]:
# Or we can check it using pyvis like above

main.show()

main.html


#### Using Program calls

Using sub-programs is an important feature of HybridAGI, their are used to allow the graph programs to scale the numerous steps for long tasks, help the developer encapsulate behaviors and allow the system to adapt and learn by being able to dynamically call or modify them.

In [5]:

clarify_objective = gp.GraphProgram(
    name="clarify_objective",
    description="Clarify the objective by asking question to the user",
)

clarify_objective.add(gp.Decision(
    id = "is_anything_unclear",
    purpose = "Check if the Objective is unclear",
    question = "Is the Objective still unclear?",
))

clarify_objective.add(gp.Action(
    id = "clarify",
    purpose = "Ask question to clarify the user request",
    tool = "AskUser",
    prompt = "Pick one question to clarify the Objective",
))

clarify_objective.add(gp.Action(
    id = "refine_objective",
    purpose = "Refine the question",
    tool = "UpdateObjective",
    prompt = "Refine the Objective",
))

clarify_objective.connect("start", "is_anything_unclear")
clarify_objective.connect("is_anything_unclear", "clarify", label="Clarify")
clarify_objective.connect("is_anything_unclear", "end", label="Answer")
clarify_objective.connect("clarify", "refine_objective")
clarify_objective.connect("refine_objective", "end")

clarify_objective.build()

main = gp.GraphProgram(
    name="main",
    description="The main program",
)

main.add(gp.Program(
    id = "clarify_objective",
    purpose = "Clarify the user objective if needed",
    program = "clarify_objective"
))

main.add(gp.Action(
    id = "answer",
    purpose = "Answer the objective's question",
    tool = "Speak",
    prompt = "Answer the Objective's question",
))

main.connect("start", "clarify_objective")
main.connect("clarify_objective", "answer")
main.connect("answer", "end")

main.build()


#### Note on graph program validation

Although we verify the structure of the program, we cannot verify if the name of tool used is accurate or if the program referenced is correct outside of the execution environment. This implies that you should be cautious in using the appropriate names, otherwise, the interpreter Agent will generate an error when it encounters the problematic step.

#### Loading from Cypher

You can also load your programs from cypher, this option is interresting for people with Cypher experience or to load many programs as once.

In [ ]:
# TODO